In [ ]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import os
from langchain.embeddings import HuggingFaceEmbeddings  # Correct class from HuggingFace
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings, VectorStoreIndex, PromptTemplate, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama  # Assuming you corrected the import here

# Loading documents from directory

import os
from elasticsearch import Elasticsearch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import VectorStoreIndex, PromptTemplate, SimpleDirectoryReader
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.ollama import Ollama
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:

# Load the embedding model
def load_embedding_model(model_name="sentence-transformers/all-mpnet-base-v2", device="cuda"):
    model_kwargs = {"device": device}
    encode_kwargs = {"normalize_embeddings": True}
    return HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)


os.environ["OPENAI_API_KEY"] = "sk-proj-besqHNvavvOogW1gXihmT3BlbkFJwLDqlxM3BE7iX1P9D40o"


# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

# gpt-3
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(
    llm=gpt35, verbose=True
)


# Load documents from a directory and index them in Elasticsearch and VectorStoreIndex
def load_and_index_documents(directory):
    reader = SimpleDirectoryReader(directory, recursive=True)
    docs = reader.load_data()
    # Example to check the structure of docs
    if docs and isinstance(docs[0], dict) and 'id' in docs[0] and 'content' in docs[0]:
        index_documents(docs)
    else:
        logger.error("Document structure not as expected.")
    return docs

docs = load_and_index_documents("/mnt/c/Users/edeep/RAG/RAG_Codebase/project3_se-final_with_openai/project3_se-final_with_openai")
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

index = VectorStoreIndex.from_documents(docs)
index.embedding_model = embed_model

# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine

query_engine = index.as_query_engine(llm=gpt35)
query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
)
response_gpt35 = query_engine.query(
  "What all services are there in the project and what is the purpose of each service aned what does vendor service do?",
)

print(str(response_gpt35))

